In [5]:

# Student Name : Enea Grahovac
# Cohort       : Valencia

################################################################################
# Import Packages
################################################################################


# importing packages
import pandas as pd # data science essentials
import numpy as np
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # enhanced data visualization
from sklearn.model_selection import train_test_split # train/test split
from sklearn.linear_model import Lasso # Lasso Regression
from sklearn.model_selection import cross_val_score
import sklearn.linear_model

# In[ ]:


################################################################################
# Load Data
################################################################################

chef_df     = pd.read_excel('Apprentice_Chef_Dataset.xlsx')

# In[ ]:

#chef = pd.read_excel (r'/Users/eneagrahovac/Desktop/Machine Learning /Apprentice_Chef_Dataset.xlsx')

#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)

#print (chef) # 29 predicting variables 
#chef.head(n=5)
#chef.info()
#chef.describe().round(2)
#chef.isnull().sum() #47 missing values in FAMILY_NAME column

#print (chef) # 29 predicting variables
#chef.head(n=5)
#chef.info()
#chef.describe().round(2)
#chef.isnull().sum() #47 missing values in FAMILY_NAME column

# looping over columns with missing values
#for col in chef:

    # creating columns with 1s if missing and 0 if not
    #if chef[col].isnull().astype(int).sum() > 0:
        #chef['m_'+col] = chef[col].isnull().astype(int)


    # displaying the first five rows of the DataFrame
#chef.head(n = 5)
    #creating an imputation value
#fill = 0

    # imputing variable with missing value 'FAMILY NAME'
#chef['FAMILY_NAME'] = chef['FAMILY_NAME'].fillna(fill)

# building a base model

# blueprinting a model type
#revenue_meals = smf.ols(formula = """REVENUE ~ chef['TOTAL_MEALS_ORDERED']""",
                        #data = chef)


# telling Python to run the data through the blueprint
#results = revenue_meals.fit()


# printing the results
#print(results.summary

################################################################################
# Feature Engineering, Variable Selection and (optional) Dataset Standardization
################################################################################

email_lst = []

# looping over each email address
for index, col in chef_df.iterrows():
    
    # splitting email domain at '@'
    split_email = chef_df.loc[index, 'EMAIL'].split(sep = '@')
    
    # appending placeholder_lst with the results
    email_lst.append(split_email)
    

# converting multipurpose_lst into a DataFrame 
email_df = pd.DataFrame(email_lst)

# Create the domain column in df
email_df.columns = ['id', 'email_domain']

# concatenating personal_email_domain with original_df
chef_df = pd.concat([chef_df, email_df.loc[:,'email_domain']], axis = 1)

# Classifying the email domains

professional_email_domains = ['@mmm.com','@amex.com','@apple.com','@boeing.com','@caterpillar.com','@chevron.com',
                              '@cisco.com','@cocacola.com','@disney.com','@dupont.com','@exxon.com','@ge.org',
                              '@goldmansacs.com','@homedepot.com','@ibm.com','@intel.com','@jnj.com','@jpmorgan.com',
                              '@mcdonalds.com','@merck.com','@microsoft.com','@nike.com','@pfizer.com','@pg.com',
                              '@travelers.com','@unitedtech.com','@unitedhealth.com','@verizon.com','@visa.com',
                              '@walmart.com']

personal_email_domains     = ['@gmail.com', '@yahoo.com', '@protonmail.com']

junk_email_domains         = ['@me.com','@aol.com','@hotmail.com','@live.com','@msn.com','@passport.com']

# multipurpose list
email_lst = []


# looping to group observations by domain type
for domain in chef_df ['email_domain']:
        if '@'+ domain in professional_email_domains:
            email_lst.append('professional')
            
        elif '@' + domain in personal_email_domains:
            email_lst.append('personal')

        elif '@' + domain in junk_email_domains:
            email_lst.append('junk')
            
        else:
            print('Unknown')

# concatenating with CHEF_DF
chef_df ['domain_group'] = pd.Series(email_lst)

# Creating dummies before dropping COLUMNS
one_hot_email= pd.get_dummies(chef_df['domain_group'])
chef_df = chef_df.drop('domain_group', axis = 1)

chef_df=chef_df.join([one_hot_email])

# THRESHOLD

total_meals_ordered_hi       =260
total_meals_ordered_low      =20
unique_meals_purch_hi        =10
contact_cs_hi                =10
avg_time_per_visit_hi        =260
cancellation_before_noon_hi  =4
cancellation_after_noon_hi   =0.5
weekly_plan_low              =0.5
weekly_plan_hi               =14
early_deliveries_hi          =0.5
late_deliveries_hi           =6
avg_prep_vid_time_hi         =290
largest_order_size_hi        =6
master_class_attended_hi     =0.5
photos_viewed_low            =0.5
photos_viewed_hi             =399
sal_hi                       =2301


# CREATING New columns

chef_df['CONTACT_C_SERVICE_OUT'] = 0
condition = chef_df.loc[0:,'CONTACT_C_SERVICE_OUT'][chef_df['CONTACTS_W_CUSTOMER_SERVICE'] > contact_cs_hi]
chef_df['CONTACT_C_SERVICE_OUT'].replace(to_replace = condition, value = 1, inplace = True)

chef_df['S_OUT'] = 0
condition = chef_df.loc[0:,'S_OUT'][chef_df['REVENUE'] > sal_hi]
chef_df['S_OUT'].replace(to_replace = condition, value = 1, inplace = True)


# X and Y definition

x_variables = [
    "TOTAL_MEALS_ORDERED",
    "UNIQUE_MEALS_PURCH",
    "CONTACTS_W_CUSTOMER_SERVICE",
    "PRODUCT_CATEGORIES_VIEWED",
    "AVG_TIME_PER_SITE_VISIT",
    "CANCELLATIONS_AFTER_NOON",
    "TASTES_AND_PREFERENCES",
    "MOBILE_LOGINS","PC_LOGINS",  
    "FOLLOWED_RECOMMENDATIONS_PCT",
    "AVG_PREP_VID_TIME",
    "LARGEST_ORDER_SIZE",
    "MASTER_CLASSES_ATTENDED",
    "MEDIAN_MEAL_RATING",
    "AVG_CLICKS_PER_VISIT",
    "TOTAL_PHOTOS_VIEWED",
    "CONTACT_C_SERVICE_OUT",
    "S_OUT",
   ]

# Independent variables
independent_data   = chef_df.loc[ : , x_variables]


# DEPENDENT variable
target_data = chef_df.loc[ : , 'REVENUE']

# In[ ]:


# Train/Test Split


X_train, X_test, y_train, y_test = train_test_split(
            independent_data,
            target_data,
            test_size = 0.25,
            random_state = 927)


# In[ ]:

# Final Model (instantiate, fit, and predict)


# INSTANTIATING a Lasso model object

lasso_reg = sklearn.linear_model.Lasso()

# FITTING to the training data

lasso_fit=lasso_reg.fit(X_train, y_train)

# PREDICTING on new data
lasso_reg_pred = lasso_reg.predict(X_test)


# SCORING the results
print('Training Score:', lasso_reg.score(X_train, y_train).round(3))
print('Testing Score:',  lasso_reg.score(X_test, y_test).round(3))


# In[ ]:


################################################################################
# Final Model Score (score)
################################################################################

# use this space to score your final model on the testing set
# MAKE SURE TO SAVE YOUR TEST SCORE AS test_score
# Example: test_score = final_model.score(X_test, y_test)

test_score = lasso_reg.score(X_test, y_test).round(3)


Training Score: 0.807
Testing Score: 0.799
